# **5.1**

In [83]:
from itertools import combinations
from functools import cmp_to_key
import numpy as np
from itertools import combinations, product

In [84]:
def _generate_matrix_row(bit_reversed_vectors, subset_indices):
    """
    Генерирует строку матрицы на основе битовых векторов и заданного подмножества индексов.

    :param bit_reversed_vectors: Список битовых векторов, представленных в обратном порядке.
    :param subset_indices: Индексы, которые определяют подмножество элементов для вычисления.
    :return: Список, представляющий строку матрицы, где каждый элемент вычисляется как
             произведение элементов вектора, соответствующих заданным индексам, по модулю 2.
    """
    row = [(np.prod([(x + 1) for i, x in enumerate(vector) if i in subset_indices]) % 2) for vector in bit_reversed_vectors]
    return row


In [85]:
def _compare_rows(row_a, row_b):
    """
    Функция сравнения для сортировки строк на основе старшего отличающегося бита.

    :param row_a: Первая строка для сравнения (список или массив).
    :param row_b: Вторая строка для сравнения (список или массив).
    :return:
        - Положительное значение, если старший отличающийся бит в row_a больше, чем в row_b.
        - Отрицательное значение, если старший отличающийся бит в row_a меньше, чем в row_b.
        - 0, если строки равны (все биты совпадают).
    """
    for i in range(len(row_a) - 1, -1, -1):
        if row_a[i] != row_b[i]:
            return row_a[i] - row_b[i]
    return 0


In [86]:
def _sort_matrix_rows(matrix, subset_counts):
    """
    Сортирует строки матрицы в каждой группе подмножеств.

    :param matrix: Исходная матрица, строки которой необходимо отсортировать (двумерный массив или список).
    :param subset_counts: Список, содержащий количество строк в каждой группе подмножеств.
    :return: Новая матрица, в которой строки отсортированы в каждой группе подмножеств
             на основе старшего отличающегося бита.
    """
    sorted_matrix = []
    start_index = 0
    for count in subset_counts:
        end_index = start_index + count
        subset_rows = list(matrix[start_index:end_index])
        subset_rows.sort(key=cmp_to_key(_compare_rows))
        sorted_matrix.extend(subset_rows)
        start_index = end_index
    return np.array(sorted_matrix, dtype=int)


In [87]:
def generate_reed_muller_matrix(r, m):
    """
    Генерирует порождающую матрицу Рида-Маллера порядка r и длины m.

    :param r: Порядок кода Рида-Маллера, определяющий количество исправляемых ошибок.
    :param m: Длина кодового слова, определяющая размерность матрицы.
    :return: Порождающая матрица Рида-Маллера в виде двумерного массива (numpy array).

    Эта функция создает порождающую матрицу для кода Рида-Маллера, используя битовые векторы,
    формируя все возможные подмножества индексов и генерируя строки матрицы на основе этих подмножеств.
    Строки матрицы сортируются по старшему отличающемуся биту для упрощения дальнейшей обработки.
    """
    bit_reversed_vectors = [list(map(int, f"{i:0{m}b}"[::-1])) for i in range(2**m)]

    all_subsets = []
    subset_lengths = []
    for i in range(r + 1):
        subsets = list(combinations(range(m), i))
        all_subsets.extend(subsets)
        subset_lengths.append(len(subsets))

    matrix = np.array([_generate_matrix_row(bit_reversed_vectors, subset) for subset in all_subsets], dtype=int)

    return _sort_matrix_rows(matrix, subset_lengths)


In [88]:
r, m = 2, 4
G = generate_reed_muller_matrix(r, m)
print(G)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]


# **5.2**

In [89]:
def sort_indices_for_majority(m, r):
    """
    Сортирует индексы для мажоритарного декодирования.

    :param m: Общее количество индексов.
    :param r: Размер подмножества для мажоритарного декодирования.
    :return: Массив отсортированных комбинаций индексов.
    """
    indices = range(m)
    comb_list = list(combinations(indices, r))

    if comb_list:
        comb_list.sort(key=lambda x: len(x))

    return np.array(comb_list, dtype=int)

In [90]:
def calculate_f_value(vector, subset):
    """
    Вычисляет значение функции f для заданного вектора и подмножества индексов.

    :param vector: Входной бинарный вектор.
    :param subset: Подмножество индексов для вычисления.
    :return: Значение функции f.
    """
    return np.prod([(vector[index] + 1) % 2 for index in subset])

In [91]:
def create_binary_matrix(cols):
    """
    Генерирует бинарную матрицу с заданным количеством столбцов.

    :param cols: Количество столбцов в бинарной матрице.
    :return: Список всех возможных бинарных векторов длины cols.
    """
    return list(product([0, 1], repeat=cols))

In [92]:
def construct_vector_V(subset, num_cols):
    """
    Формирует вектор V_I на основе заданного подмножества индексов.

    :param subset: Подмножество индексов.
    :param num_cols: Общее количество столбцов.
    :return: Вектор V_I.
    """
    if len(subset) == 0:
        return np.ones(2 ** num_cols, int)
    else:
        v_vector = []
        for binary_vector in create_binary_matrix(num_cols):
            f_value = calculate_f_value(binary_vector, subset)
            v_vector.append(f_value)
        return v_vector

In [93]:
def construct_vector_H(I, m):
    """
    Формирует вектор H для заданного подмножества индексов.

    :param I: Подмножество индексов.
    :param m: Общее количество индексов.
    :return: Список бинарных векторов, соответствующих подмножеству I.
    """
    return [word for word in create_binary_matrix(m) if calculate_f_value(word, I) == 1]

In [94]:
def find_complement(I, m):
    """
    Формирует комплиментарное множество для заданного подмножества.

    :param I: Подмножество индексов.
    :param m: Общее количество индексов.
    :return: Комплиментарное множество индексов.
    """
    return [i for i in range(m) if i not in I]

In [95]:
def calculate_f_t_value(words, I, t):
    """
    Вычисляет значение функции f_t для заданных слов и подмножества индексов.

    :param words: Список бинарных слов.
    :param I: Подмножество индексов.
    :param t: Бинарный вектор, используемый для вычисления.
    :return: Значение функции f_t.
    """
    return np.prod([(words[j] + t[j] + 1) % 2 for j in I])

In [96]:
def construct_V_I_t(I, m, t):
    """
    Формирует вектор V_I_t на основе подмножества индексов и вектора t.

    :param I: Подмножество индексов.
    :param m: Общее количество индексов.
    :param t: Бинарный вектор, используемый для вычисления.
    :return: Вектор V_I_t.
    """
    if not I:
        return np.ones(2 ** m, dtype=int)
    return [calculate_f_t_value(word, I, t) for word in create_binary_matrix(m)]

In [97]:
def majority_decoding_algorithm(w, r, m, size):
    """
    Выполняет мажоритарное декодирование для исправления ошибок в бинарном коде.

    :param w: Входное бинарное слово с ошибками, которое необходимо декодировать.
    :param r: Максимальное количество ошибок, которые могут быть исправлены.
    :param m: Общее количество индексов (длина кодового слова).
    :param size: Размер массива Mi, который будет содержать результаты декодирования.
    :return: Массив Mi, содержащий исправленное слово, или None, если декодирование не удалось.
    """
    w_r = w.copy()
    Mi = np.zeros(size, dtype=int)
    max_weight = 2 ** (m - r - 1) - 1
    index = 0

    def process_counts(zeros_count, ones_count):
        nonlocal index
        if zeros_count > max_weight and ones_count > max_weight:
            return True

        if zeros_count > (2 ** (m - r - 1)):
            Mi[index] = 0
            index += 1

        if ones_count > (2 ** (m - r - 1)):
            Mi[index] = 1
            index += 1
            V = construct_vector_V(J, m)
            w_r[:] = (w_r + V) % 2

        return False

    while True:
        for J in sort_indices_for_majority(m, r):
            zeros_count, ones_count = 0, 0

            for t in construct_vector_H(J, m):
                komplement = find_complement(J, m)
                V = construct_V_I_t(komplement, m, t)
                c = np.dot(w_r, V) % 2

                if c == 0:
                    zeros_count += 1
                else:
                    ones_count += 1

            if process_counts(zeros_count, ones_count):
                return

        if r > 0:
            if len(w_r) < max_weight:
                for J in sort_indices_for_majority(m, r + 1):
                    Mi[index] = 0
                    index += 1
                break
            r -= 1
        else:
            break

    return Mi[::-1]

In [98]:
def create_word_with_errors(G, error_count):
    """
    Генерирует бинарное слово с заданным количеством ошибок.

    :param G: Генерационная матрица, используемая для кодирования исходного сообщения.
    :param error_count: Количество ошибок, которые необходимо ввести в закодированное слово.
    :return: Бинарное слово с введенными ошибками.
    """
    u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
    print("Исходное сообщение: ", u)

    u = u.dot(G) % 2
    mistake_positions = np.random.choice(len(u), size=error_count, replace=False)
    u[mistake_positions] = (u[mistake_positions] + 1) % 2

    return u


# **5.3**

In [99]:
# Эксперимент для однократной ошибки
Err = create_word_with_errors(G, 1)
print("Слово с однократной ошибкой:", Err)
print()


Исходное сообщение:  [1 0 0 0 1 1 0 0 0 1 1]
Слово с однократной ошибкой: [1 0 0 0 1 1 0 1 0 1 1 1 1 0 0 1]



In [100]:
Decoded_word = majority_decoding_algorithm(Err, 2, 4, len(G))
if Decoded_word is None:
    print("\nНеобходима повторная отправка сообщения")
else:
    print("Исправленное слово:", Decoded_word)
    V2 = Decoded_word.dot(G) % 2
    print("Результат умножения исправленного слова на матрицу G:", V2)


Исправленное слово: [1 1 0 0 0 1 1 0 0 0 1]
Результат умножения исправленного слова на матрицу G: [1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1]


In [101]:
# Эксперимент для двукратной ошибки
Err = create_word_with_errors(G, 2)
print("Слово с двукратной ошибкой:", Err)



Исходное сообщение:  [1 0 0 0 1 1 0 0 0 1 1]
Слово с двукратной ошибкой: [1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1]


In [102]:
Decoded_word = majority_decoding_algorithm(Err, 2, 4, len(G))
if Decoded_word is None:
   print("\nНеобходима повторная отправка сообщения")
else:
   print("Исправленное слово:", Decoded_word)
   V2 = Decoded_word.dot(G) % 2
   print("Результат умножения исправленного слова на матрицу G:", V2)


Необходима повторная отправка сообщения
